In [1]:
%matplotlib inline
from __future__ import print_function, division, absolute_import

import numpy as np
import matplotlib.pyplot as plt
from pyuvdata import UVData

# Partial Reading
UVH5 supports "select-on-read" type operations like miriad and uvdata. It also allows for reading in just the metadata from a file without actually touching the data.

In [2]:
# read in miriad file to convert to uvh5
uvd = UVData()
infile = '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_1/2458098/zen.2458098.12552.xx.HH.uvOCRS'
outfile = '/lustre/aoc/projects/hera/plaplant/test/uvh5_partial_io/zen.2458098.12552.xx.HH.h5'
uvd.read_miriad(infile)
uvd.write_uvh5(outfile, clobber=True)

Mean of empty slice.
invalid value encountered in double_scalars


File exists; clobbering


In [3]:
# read in just the file metadata
uvd2 = UVData()
uvd2.read_uvh5(outfile, read_data=False)

# print out some metadata
print("Ntimes: ", uvd2.Ntimes)
print("Nfreqs: ", uvd2.Nfreqs)
print("Nants_data: ", uvd2.Nants_data)
print("uvw_array: ", uvd2.uvw_array)
print("ant_1_array: ", uvd2.ant_1_array)

# try to print data array
print("data_array: ", uvd2.data_array)

Ntimes:  60
Nfreqs:  1024
Nants_data:  52
uvw_array:  [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.46078427e+01  5.57885268e-02 -9.77542666e-03]
 [ 2.92156851e+01  1.11577054e-01 -1.95842754e-02]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.46078392e+01  5.57866607e-02  1.41013574e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]
ant_1_array:  [  0   0   0 ... 142 142 143]
data_array:  None


In [4]:
# perform select-on-read, like uvfits or miriad
# read certain baselines (i.e., waterfalls)
bl = (0, 1, 'xx')
uvd2.read_uvh5(outfile, bls=[bl])

# show updated metadata
print("Nants_data: ", uvd2.Nants_data)
print("data_array shape: ", uvd2.data_array.shape)

Nants_data:  2
data_array shape:  (60, 1, 1024, 1)


In [5]:
# read certain frequencies
freq_chans = np.arange(512)
uvd2.read_uvh5(outfile, freq_chans=freq_chans)

# show updated metadata
print("Nants_data: ", uvd2.Nants_data)
print("Nfreqs: ", uvd2.Nfreqs)
print("data_array shape: ", uvd2.data_array.shape)

Nants_data:  52
Nfreqs:  512
data_array shape:  (82680, 1, 512, 1)


In [6]:
# read certain times
uvd2.read_uvh5(outfile, read_data=False)
times = np.unique(uvd2.time_array)
Ntimes = len(times)
uvd2.read_uvh5(outfile, times=times[:(Ntimes // 2)])

# show updated metadata
print("Ntimes: ", uvd2.Ntimes)
print("data_array shape: ", uvd2.data_array.shape)

Ntimes:  30
data_array shape:  (41340, 1, 1024, 1)


In [7]:
# select along multiple axes
uvd2.read_uvh5(outfile, times=times[:(Ntimes // 2)], freq_chans=freq_chans)

# show updated metadata
print("Ntimes: ", uvd2.Ntimes)
print("Nfreqs: ", uvd2.Nfreqs)
print("data_array shape: ", uvd2.data_array.shape)

Ntimes:  30
Nfreqs:  512
data_array shape:  (41340, 1, 512, 1)


# Partial Writing
New for UVH5, we are now able to write data out in parts. It requires first initializing an empty file using an object that has the proper metadata of the final object. Then, the user writes the data out in parts, without needing to save all of the intermediate data in memory.

In [8]:
# create a new file on disk from object's metadata
uvd2 = UVData()
uvd2.read_uvh5(outfile, read_data=False)

partfile = '/lustre/aoc/projects/hera/plaplant/test/uvh5_partial_io/zen.2458098.12552.xx.HH.partial.h5'
uvd2.initialize_uvh5_file(partfile, clobber=True)

File exists; clobbering


In [9]:
# read in the lower half of the band and apply different scalings to both
Nfreqs = uvd2.Nfreqs
Hfreqs = Nfreqs // 2
freq_inds1 = np.arange(Hfreqs)
freq_inds2 = np.arange(Hfreqs, Nfreqs)
uvd3 = UVData()
uvd3.read_uvh5(outfile, freq_chans=freq_inds1)
data_array = 0.5 * uvd3.data_array
flag_array = uvd3.flag_array
nsample_array = uvd3.nsample_array

uvd2.write_uvh5_part(partfile, data_array, flag_array, nsample_array, freq_chans=freq_inds1)

uvd3.read_uvh5(outfile, freq_chans=freq_inds2)
data_array = 2.0 * uvd3.data_array
flag_array = uvd3.flag_array
nsample_array = uvd3.nsample_array

uvd2.write_uvh5_part(partfile, data_array, flag_array, nsample_array, freq_chans=freq_inds2)